### 2. Movie 노드 : 임베딩 필드 추가 및 벡터 인덱스 생성
2.1 임베딩 모델 초기화

In [1]:
import os
from dotenv import load_dotenv
from langchain_google_genai import GoogleGenerativeAIEmbeddings

load_dotenv()

embeddings = GoogleGenerativeAIEmbeddings(model="models/text-embedding-004")

In [2]:
from langchain_neo4j import Neo4jGraph

#LangChain 도구 활용 - DB연결 객체 초기화
graph = Neo4jGraph(
    url = os.getenv("NEO4J_URI"),
    username = os.getenv("NEO4J_USERNAME"),
    password = os.getenv("NEO4J_PASSWORD"),
)

In [3]:
# 테스트 쿼리 실행
cypher_query="""
MATCH (n:Movie)
RETURN COUNT(n) AS Movie_Count
"""

graph.query(cypher_query)

[{'Movie_Count': 4803}]

### 2.2 벡터 인덱스 생성

In [4]:
create_vector_index_query="""

// 영화 콘텐츠 임베딩을 위한 벡터 인덱스 생성
CREATE VECTOR INDEX movie_content_embeddings IF NOT EXISTS

// Movie 노드의 content_embedding 속성에 인덱스 적용
FOR (m:Movie) ON m.content_embedding

// 벡터 인덱스 설정 옵션
OPTIONS {
    indexConfig: {
        `vector.dimensions`: 768,
        `vector.similarity_function`: 'cosine'
    }
}
"""

graph.query(create_vector_index_query)

[]

In [5]:
#벡터 인덱스 확인
check_vector_index_query="""SHOW VECTOR INDEXES"""

vector_indexes = graph.query(check_vector_index_query)
for index in vector_indexes:
    print(f"Index Name: {index['name']}")
    print(f"Type: {index['type']}")
    print(f"Property Key : {index['properties']}")
    print("-"*40)

Index Name: movie_content_embeddings
Type: VECTOR
Property Key : ['content_embedding']
----------------------------------------


### 2.3 임베딩 생성 및 저장

In [6]:
#영화 제목과 줄거리 가져오기
movies_query="""
MATCH (m:Movie)
WHERE m.title IS NOT NULL
RETURN m.id AS id, m.title AS title, m.overview AS overview, m.tagline AS tagline
"""

movies = graph.query(movies_query)

#배치 크기 설정
BATCH_SIZE = 100

#임베딩 생성 및 저장 (배치처리)
for i in range(0, len(movies), BATCH_SIZE):
    batch = movies[i:i+BATCH_SIZE]
    batch_texts =[]
    batch_ids = []

    #배치 데이터 준비
    for movie in batch:
        # overview와 tagline을 "\n\n"으로 결합
        content_text = f"{movie['title']}"
        if movie['tagline']:
            content_text+=f"\n\n{movie['tagline']}"
        if movie['overview']:
            content_text += f"\n\n{movie['overview']}"

        if content_text.strip():
            batch_texts.append(content_text)
            batch_ids.append(movie['id'])
    
    try:
        if batch_texts:
            #배치 단위로 googleAI 임베딩 생성
            batch_embeddings=embeddings.embed_documents(batch_texts)

            #UNWIND를 사용한 배치 업데이트
            batch_data =[{"id": article_id, "embedding":embedding_vector}
                        for article_id, embedding_vector in zip(batch_ids, batch_embeddings)]

            batch_update_query = """
            UNWIND $batch AS item
            MATCH (m:Movie {id: item.id})
            CALL db.create.setNodeVectorProperty(m, 'content_embedding', item.embedding)
            RETURN count(m) as updated
            """

            result = graph.query(batch_update_query, params={"batch": batch_data})
            print(f"배치 처리 완료: {i+1}~{min(i+len(batch_texts), len(movies))} / {len(movies)}, 업데이트됨: {result[0]['updated']}")
    
    except Exception as e:
        print(f"배치 임베딩 생성 실패 (배치 인덱스 {i}: {str(e)})")

print(f"영화 임베딩 업데이트 완료!! 총 {len(movies)}개 처리")


배치 처리 완료: 1~100 / 4803, 업데이트됨: 100
배치 처리 완료: 101~200 / 4803, 업데이트됨: 100
배치 처리 완료: 201~300 / 4803, 업데이트됨: 100
배치 처리 완료: 301~400 / 4803, 업데이트됨: 100
배치 처리 완료: 401~500 / 4803, 업데이트됨: 100
배치 처리 완료: 501~600 / 4803, 업데이트됨: 100
배치 처리 완료: 601~700 / 4803, 업데이트됨: 100
배치 처리 완료: 701~800 / 4803, 업데이트됨: 100
배치 처리 완료: 801~900 / 4803, 업데이트됨: 100
배치 처리 완료: 901~1000 / 4803, 업데이트됨: 100
배치 처리 완료: 1001~1100 / 4803, 업데이트됨: 100
배치 처리 완료: 1101~1200 / 4803, 업데이트됨: 100
배치 처리 완료: 1201~1300 / 4803, 업데이트됨: 100
배치 처리 완료: 1301~1400 / 4803, 업데이트됨: 100
배치 처리 완료: 1401~1500 / 4803, 업데이트됨: 100
배치 처리 완료: 1501~1600 / 4803, 업데이트됨: 100
배치 처리 완료: 1601~1700 / 4803, 업데이트됨: 100
배치 처리 완료: 1701~1800 / 4803, 업데이트됨: 100
배치 처리 완료: 1801~1900 / 4803, 업데이트됨: 100
배치 처리 완료: 1901~2000 / 4803, 업데이트됨: 100
배치 처리 완료: 2001~2100 / 4803, 업데이트됨: 100
배치 처리 완료: 2101~2200 / 4803, 업데이트됨: 100
배치 처리 완료: 2201~2300 / 4803, 업데이트됨: 100
배치 처리 완료: 2301~2400 / 4803, 업데이트됨: 100
배치 처리 완료: 2401~2500 / 4803, 업데이트됨: 100
배치 처리 완료: 2501~2600 / 4803, 업데이트됨: 100
배치 처리 완

In [8]:
def semantic_movie_search(search_text, top_k=5):
    """텍스트 쿼리를 받아 의미적으로 가장 유사한 영화를 반환합니다.
    
    매개변수 : 
        search_text (str) :검색할 텍스트 쿼리
        top_k (int) : 반환할 최대 결과수 (기본값: 5)
        
    반환값 : 
        list : 유사도 점수가 높은 순으로 정렬된 영화 정보 목록
    """

    #검색 텍스트의 임베딩
    query_embedding = embeddings.embed_query(search_text)

    #Neo4j 벡터 검색 쿼리 실행
    vector_search_query="""
    CALL db.index.vector.queryNodes(
        'movie_content_embeddings', //사용할 벡터 인덱스 이름
        $top_k,
        $query_embedding //검색 쿼리의 임제딩 벡터
    ) YIELD node, score //검색결과 노드와 유사도 점수 반환
    RETURN node.title AS title, //영화제목
           node.released AS released, //개봉일
           node.rating AS rating, //평점
           score AS similarity  //유사도 점수
    ORDER BY similarity DESC
    """

    results = graph.query(
        vector_search_query,
        params = {"top_k": top_k, "query_embedding": query_embedding}
    )

    #검색결과 반환 (영화제목, 개봉일, 평점, 유사도 점수 포함)
    return results


In [9]:
#임베딩 기반 의미 검색 (영어 예시)
search_text = "a Drama movie about artificial intelligence and reality"

#의미 검색 실행 
results = semantic_movie_search(search_text)

for result in results :
    print(f"{result['title']} - 유사도: {result['similarity']:.4f}, "+ f"평점: {result['rating']},"+ f"개봉: {result['released']}")

Ex Machina - 유사도: 0.8190, 평점: 7.6,개봉: 2015-01-21
Transcendence - 유사도: 0.8144, 평점: 5.9,개봉: 2014-04-16
A.I. Artificial Intelligence - 유사도: 0.8104, 평점: 6.8,개봉: 2001-06-29
Surrogates - 유사도: 0.8048, 평점: 5.9,개봉: 2009-09-24
Automata - 유사도: 0.7939, 평점: 5.6,개봉: 2014-10-09


In [10]:
#임베딩 기반 의미 검색 (한국어 예시)
search_text_kr = "인공지능과 현실에 관한 SF 영화"

#의미 검색 실행
results_kr = semantic_movie_search(search_text_kr)

for result in results_kr :
    print(f"{result['title']} - 유사도: {result['similarity']:.4f}, "+ f"평점: {result['rating']},"+ f"개봉: {result['released']}")

Groove - 유사도: 0.7634, 평점: 5.6,개봉: 2000-06-08
Jade - 유사도: 0.7500, 평점: 5.2,개봉: 1995-10-13
Street Fighter - 유사도: 0.7488, 평점: 4.1,개봉: 1994-12-22
Silver Medalist - 유사도: 0.7403, 평점: 7.4,개봉: 2009-01-20
The X Files - 유사도: 0.7402, 평점: 6.6,개봉: 1998-06-19


### 3. 하이브리드 검색

벡터검색 + 키워드 필터링

In [12]:
def hybrid_movie_search(search_text, genre=None, min_rating=None, top_k=5):
    """벡터 검색 및 키워드 필터링을 결합한 하이브리드 검색
    
    Args:
        search_text :  검색할 텍스트 쿼리
        genre : 필터링할 영화 장르 (선택적)
        min_rating : 최소 평점 기준 (선택적)
        top_k : 반환할 최대 결과 수

    Returns : 
        필터링 된 영화 검색 결과 목록
    """
    #검색 텍스트의 임베딩 생성 - googleAI 임베딩 모델을 사용하여 텍스트를 벡터로 변환
    query_embedding = embeddings.embed_query(search_text)

    #추가 필터링 조건 구성 - 사용자가 지정한 필터에 따라 쿼리 조건 생성
    filters=[]

    if genre:
        filters.append("EXISTS {MATCH (node)-[:IN_GENRE]->(:Genre{name:$genre})}")
    
    if min_rating:
        filters.append("node.rating >= $min_rating")

    #필터 조건을 WHERE 절로 변환 - 필터가 있는 경우에만 WHERE절 추가
    where_clause =""
    if filters:
        where_clause = "WHERE "+ " AND ".join(filters)
    
    #Filter 조건 출력
    print(f"Filter 조건 : \n{where_clause}\n\n")

    #벡터 검색 쿼리 실행
    hybrid_search_query = f"""
    CALL db.index.vector.queryNodes(
        'movie_content_embeddings',
        100,                           //초기 검색 결과 수
        $query_embedding               //파라미터로 전달된 쿼리 임베딩 벡터
    ) YIELD node, score

    {where_clause} // 동적으로 생성된 필터링 조건 (장르,평점 등)

    WITH node, score
    OPTIONAL MATCH (node)-[:IN_GENRE]->(g:Genre) // 영화와 연결된 장르 찾기

    RETURN node.title AS title,
           node.released AS released,
           node.rating AS rating,
           node.tagline AS tagline,
           node.overview AS overview,
           collect(g.name) AS genres,
           score AS similarity
    
    ORDER BY similarity DESC
    LIMIT $top_k
    """
    # 쿼리 파라미터 설정 - 동적으로 필요한 파라미터만 포함
    params={
        "query_embedding": query_embedding,
        "top_k" : top_k
    }

    # 선택적 파라미터 추가 - 필터가 지정된 경우에만 해당 파라미터 추가
    if genre :
        params["genre"] = genre
    if min_rating:
        params["min_rating"] = min_rating
    
    # Neo4j 데이터베이스에 쿼리 실행 및 결과 반환
    results = graph.query(hybrid_search_query, params = params)

    return results

# 하이브리드 검색 테스트
hybrid_results = hybrid_movie_search(
    "가족간의 사랑과 신뢰 회복을 주제로 한 영화",
    genre="Drama",
    min_rating=7.0,
    top_k=5
)

# 검색 결과 출력 - 영화 제목, 유사도 점수, 평점, 장르 정보 표시
for result in hybrid_results:
    print(f"{result['title']} - 유사도: {result['similarity']:.4f}"+ f"평점: {result['rating']}, 장르: {', '.join(result['genres'])}")
    print(f"---- 태그라인: {result['tagline']}")
    print(f"---- 개요: {result['overview']}")
    print()

Filter 조건 : 
WHERE EXISTS {MATCH (node)-[:IN_GENRE]->(:Genre{name:$genre})} AND node.rating >= $min_rating


Silver Medalist - 유사도: 0.7427평점: 7.4, 장르: Drama, Action, Comedy, Adventure, Foreign
---- 태그라인: None
---- 개요: An action-adventure story focused on the lives of express deliverymen, traffic cops and lonely beauties.

Chiamatemi Francesco - Il Papa della gente - 유사도: 0.7383평점: 7.3, 장르: Drama
---- 태그라인: None
---- 개요: None

Madadayo - 유사도: 0.7381평점: 7.5, 장르: Drama
---- 태그라인: None
---- 개요: This film tells the story of professor Uehida Hyakken-sama (1889-1971), in Gotemba, around the forties. He was a university professor until an air raid, when he left to become a writer and has to live in a hut. His mood has hardly changed, not by the change nor by time.

Children of Heaven - 유사도: 0.7354평점: 7.8, 장르: Drama, Comedy, Family
---- 태그라인: A Little Secret...Their Biggest Adventure!
---- 개요: Zohre's shoes are gone; her older brother Ali lost them. They are poor, there are no shoes for Zohre u

In [13]:
# 하이브리드 검색 테스트 - genre 필터 제외
hybrid_results = hybrid_movie_search(
    "친구들의 우정을 그린 영화",
    min_rating=7.0,
    top_k=5
)

# 검색 결과 출력 - 영화 제목, 유사도 점수, 평점, 장르 정보 표시
for result in hybrid_results:
    print(f"{result['title']} - 유사도: {result['similarity']:.4f}"+ f"평점: {result['rating']}, 장르: {', '.join(result['genres'])}")
    print(f"---- 태그라인: {result['tagline']}")
    print(f"---- 개요: {result['overview']}")
    print()

Filter 조건 : 
WHERE node.rating >= $min_rating


Silver Medalist - 유사도: 0.7543평점: 7.4, 장르: Drama, Action, Comedy, Adventure, Foreign
---- 태그라인: None
---- 개요: An action-adventure story focused on the lives of express deliverymen, traffic cops and lonely beauties.

Open Secret - 유사도: 0.7442평점: 7.0, 장르: Thriller, Mystery, Crime
---- 태그라인: The Pull-No-Punch drama of men chained together by hate!
---- 개요: A couple discovers that their friend has gone missing. Their investigation leads them to believe that anti-semites are behind the disappearance.

Oldboy - 유사도: 0.7433평점: 8.0, 장르: Drama, Thriller, Action, Mystery
---- 태그라인: 15 years of imprisonment, five days of vengeance
---- 개요: With no clue how he came to be imprisoned, drugged and tortured for 15 years, a desperate businessman seeks revenge on his captors.

Madadayo - 유사도: 0.7401평점: 7.5, 장르: Drama
---- 태그라인: None
---- 개요: This film tells the story of professor Uehida Hyakken-sama (1889-1971), in Gotemba, around the forties. He was a univ